## ML project 2

### Notes

- train model for each expert
- 0.5 'missing gender' somehing else than male/ female
- model ohne metadata wie age gender etc, just the coughs
- person coughs --> want to tell wether wet or dry
- dont use other expert information

Features:

- Columns C-BQ: Computed features in the time and frequency domain
- Column BR: The expert that reviwed the cough
- Columns BS-BV: Metadata features provided by users
    - Some values were not provided by users and were set to average values for each feature
- The 'SNR' feature exists for non-segmented coughs but not segmented coughs

In [3]:
import pandas as pd
import numpy as np
import IPython.display
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import plot_roc_curve, auc, roc_curve

from imblearn.over_sampling import SMOTE

from sklearn import svm
from sklearn.model_selection import StratifiedKFold

#import src.utils
from src.utils.preprocessing import import_data, standardize, dummy_code

from src.utils.model_helpers import roc_w_cross_val

import importlib

In [12]:
# to reload modules
importlib.reload(src.utils)

NameError: name 'src' is not defined

In [4]:
dir(src.utils.preprocessing)

NameError: name 'src' is not defined

In [5]:
DATA_PATH = '../../data'

## Load Data

In [6]:
# Load data
# Labels: # 1 - Wet Cough # 0 - Dry Cough

coarse_data, coarse_labels = import_data(DATA_PATH, segmentation_type = 'coarse', is_user_features=True)
fine_data, fine_labels = import_data(DATA_PATH, segmentation_type = 'fine', is_user_features=True)
no_data, no_labels = import_data(DATA_PATH, segmentation_type = 'no', is_user_features=True)

#display(coarse_data.head(2), fine_data.head(2), no_data.head(2))
#display(coarse_labels.head(2), fine_labels.head(2), no_labels.head(2))

1. Data Preprocessing

3.Deal with missing metadata appropriately

2.Perform exploratory data analysis and feature engineering (ex. examine theeffects of normalizing features, recursive feature elimination)

4
2.Make sure that data from a single subject does not end up in both thetraining and validation groups


### Normalisation

In [7]:
# For case where is_user_features=True
X_coarse = standardize(coarse_data, 0,-3)
X_fine = standardize(fine_data, 0,-3)
X_no = standardize(no_data, 1,-3)

In [76]:
# For case where is_user_features=False
#X_coarse = standardize(coarse_data, 0, -1)
#X_fine = standardize(fine_data, 0, -1)
#X_no = standardize(no_data, 1, -1)

### Data Preprocessing

In [ ]:
# Xavi?

## Feature Engineering

### Find interesting features

In [156]:
no_data.columns

Index(['File_Name', 'EEPD50_100', 'EEPD100_150', 'EEPD150_200', 'EEPD200_250',
       'EEPD250_300', 'EEPD300_350', 'EEPD350_400', 'EEPD400_450',
       'EEPD450_500', 'EEPD500_550', 'EEPD550_600', 'EEPD600_650',
       'EEPD650_700', 'EEPD700_750', 'EEPD750_800', 'EEPD800_850',
       'EEPD850_900', 'EEPD900_950', 'EEPD950_1000', 'Zero_Crossing_Rate',
       'RMS_Power', 'Dominant_Freq', 'Spectral_Centroid', 'Spectral_Rolloff',
       'Spectral_Spread', 'Spectral_Skewness', 'Spectral_Kurtosis',
       'Spectral_Bandwidth', 'Spectral_Flatness', 'Spectral_StDev',
       'Spectral_Slope', 'Spectral_Decrease', 'MFCC_mean0', 'MFCC_mean1',
       'MFCC_mean2', 'MFCC_mean3', 'MFCC_mean4', 'MFCC_mean5', 'MFCC_mean6',
       'MFCC_mean7', 'MFCC_mean8', 'MFCC_mean9', 'MFCC_mean10', 'MFCC_mean11',
       'MFCC_mean12', 'MFCC_std0', 'MFCC_std1', 'MFCC_std2', 'MFCC_std3',
       'MFCC_std4', 'MFCC_std5', 'MFCC_std6', 'MFCC_std7', 'MFCC_std8',
       'MFCC_std9', 'MFCC_std10', 'MFCC_std11', 'MFCC_s

In [8]:
#no_data_X_tr.iloc[:,0:-5]
# possibly nice features:
     
c0 = ['EEPD50_100', 'EEPD100_150', 'EEPD150_200', 'EEPD200_250', 'EEPD250_300', 'EEPD300_350', 'EEPD350_400', 'EEPD400_450',
       'EEPD450_500', 'EEPD500_550', 'EEPD550_600', 'EEPD600_650','EEPD650_700', 'EEPD700_750', 'EEPD750_800', 'EEPD800_850',
       'EEPD850_900', 'EEPD900_950', 'EEPD950_1000', 'Cough_Length']


c1 = ['Zero_Crossing_Rate',
       'RMS_Power', 'Dominant_Freq', 'Spectral_Centroid', 'Spectral_Rolloff',
       'Spectral_Spread', 'Spectral_Skewness', 'Spectral_Kurtosis',
       'Spectral_Bandwidth', 'Spectral_Flatness', 'Spectral_StDev',
       'Spectral_Slope', 'Spectral_Decrease']

c2 = ['MFCC_mean0', 'MFCC_mean1',
       'MFCC_mean2', 'MFCC_mean3', 'MFCC_mean4', 'MFCC_mean5', 'MFCC_mean6',
       'MFCC_mean7', 'MFCC_mean8', 'MFCC_mean9', 'MFCC_mean10', 'MFCC_mean11',
       'MFCC_mean12', 'MFCC_std0', 'MFCC_std1', 'MFCC_std2', 'MFCC_std3',
       'MFCC_std4', 'MFCC_std5', 'MFCC_std6', 'MFCC_std7', 'MFCC_std8',
       'MFCC_std9', 'MFCC_std10', 'MFCC_std11', 'MFCC_std12']

c3 = ['Crest_Factor',
       'Cough_Length', 'SNR']


c4 = ['Zero_Crossing_Rate',
       'RMS_Power', 'Dominant_Freq', 'Spectral_Centroid', 'Spectral_Rolloff',
       'Spectral_Spread', 'Spectral_Skewness', 'Spectral_Kurtosis',
       'Spectral_Bandwidth', 'Spectral_Flatness', 'Spectral_StDev',
       'Spectral_Slope', 'Spectral_Decrease', 'Crest_Factor',
       'Cough_Length', 'SNR', 'PSD_225-425', 'PSD_450-550', 'PSD_1325-1600',
       'PSD_1600-2000', 'PSD_2500-2900', 'PSD_3100-3700' ]

c5 = ['PSD_225-425', 'PSD_450-550', 'PSD_1325-1600',
       'PSD_1600-2000', 'PSD_2500-2900', 'PSD_3100-3700']

c6 = c3 + c5 # best one for Linear Regression

c7 = ['Expert', 'Age',
       'Gender', 'Resp_Condition', 'Symptoms'] # metadata

# best without 

# best columns
c = ['File_Name', 'Crest_Factor', 'Cough_Length', 'SNR', 'Expert', 'Age', 'Gender', 'Resp_Condition', 'Symptoms']
cc = ['File_Name', 'Crest_Factor', 'Cough_Length', 'Expert', 'Age', 'Gender', 'Resp_Condition', 'Symptoms']

### Inspect class imbalance

In [9]:
print(no_labels['Label'].sum() / no_labels['Label'].shape[0], coarse_labels['Label'].sum() / coarse_labels['Label'].shape[0],
      fine_labels['Label'].sum() / fine_labels['Label'].shape[0])

0.2591922845087402 0.26976744186046514 0.27122271365387357


In [ ]:
# Do something about it: SMODE
#TODO
oversample = SMOTE(random_state=42)
fn_oversampled, ln_oversampled = oversample.fit_resample(fn, ln)

fn_oversampled = pd.DataFrame(fn_oversampled, columns=fn.columns)
ln_oversampled = pd.DataFrame(ln_oversampled, columns=ln.columns)

sns.countplot(x = 'Label', data=ln_oversampled)
print('Resampled dataset shape %s' % Counter(ln_oversampled["Label"]))

### Inspect Missing Data

### Dummy code categorical features

In [10]:
X_coarse = dummy_code(X_coarse, columns = ['Gender', 'Resp_Condition', 'Symptoms'])
X_fine = dummy_code(X_fine, columns = ['Gender', 'Resp_Condition', 'Symptoms'])
X_no = dummy_code(X_no, columns = ['Gender', 'Resp_Condition', 'Symptoms'])

## Train and Test data

In [11]:
#a, b, c, d = train_test_split(coarse_data, coarse_labels, random_state = 1, test_size = 0.2)

def train_test(X, y, fraction = 0.8, random_state = 1, segmentation = True):
    
    # If there are several instances of the same subject, we don't want it to be in the training and testing set
    if segmentation == True:
        X[['Subject', 'Cough']] = X['File_Name'].str.split("_", expand = True)
        y[['Subject', 'Cough']] = y['File_Name'].str.split("_", expand = True)
    
        # index for Subject
        X = X.set_index(['Subject'])
        y = y.set_index(['Subject'])
    
    train_X = X.loc[X.sample(frac = fraction, random_state=random_state).index.unique()].drop(['File_Name'], axis = 1)
    test_X = X.drop(train_X.index).drop(['File_Name'], axis = 1)
    
    train_y = y.loc[y.sample(frac = fraction, random_state=random_state).index.unique()].drop(['File_Name'], axis = 1)
    test_y = y.drop(train_y.index).drop(['File_Name'], axis = 1)
    
    return train_X, test_X, train_y, test_y


### Also build model that has no expert features in it

### Choose features

In [12]:
# Choose some features
no_data = no_data[c]
coarse_data = coarse_data[cc]
fine_data = fine_data[cc]

KeyError: "['Expert'] not in index"

In [13]:
#TODO replace by Xavis fct
random_state = 1

# Split no segmentation data
no_data_X_tr, no_data_X_te, no_labels_y_tr, no_labels_y_te = train_test(no_data, no_labels, segmentation = False)

# Divide coarse and fine data
coarse_data_X_tr, coarse_data_X_te, coarse_labels_y_tr, coarse_labels_y_te = train_test(coarse_data, coarse_labels, random_state = random_state)

fine_data_X_tr, fine_data_X_te, fine_labels_y_tr, fine_labels_y_te = train_test(fine_data, fine_labels, random_state = random_state)


KeyError: 'File_Name'

## Methoden

2. Methods aufsetzen

- ICH

1.Perform wet vs dry cough classification using state-of-the-art ML classificationalgorithms (Logistic Regression, Support Vector Machines, Linear DiscriminantAnalysis, k Nearest Neighbors, Gaussian Naive Bayes, Decision Tree, RandomForest, and eXtreme Gradient Boosting) using the computed features andsubject metadata.


6.Report meaningful scores accounting for potential label imbalance (i.e.report AUC instead of classification accuracy)

### Logistic regression

4. Model Comparison


4.Compare classification success on segmented and non-segmented coughrecordings (all pre-processing will be done by the lab)
5.Tune the model hyperparameters appropriately

3.Assess the importance of different features to the classification result by analyzing the weights of the classifier or SHAP values.

5. Deep learning -> autoencoder
4.(Bonus) Try using Deep Learning methods to perform cough classificationdirectly on the raw audio signals

5. Explainability